### YAW PITCH ROLL

![Link](./images/yaw_pitch_roll.jpg)

- 주의 : Pitch의 절대값이 커질수로 Roll 값의 신뢰도는 떨어짐

In [ ]:
import cv2
import numpy as np
from insightface.app.common import Face
from typing import Tuple, Optional

def get_yaw_pitch_roll(face: Face) -> Tuple[Optional[float], Optional[float], Optional[float]]:
    return face.pose[1], face.pose[0], face.pose[2] #yaw, pitch, roll

In [ ]:
TARGET_IMG_FILE = "cross.png"

In [ ]:
# 얼굴 인식을 위해 InsightFace를 사용하는 샘플 코드

import cv2
from insightface.app import FaceAnalysis

# FaceAnalysis 객체 초기화 (사전 학습된 모델 사용)
app = FaceAnalysis(name='buffalo_l')  # 'buffalo_l'는 사전 학습된 모델 이름입니다.
app.prepare(ctx_id=-1)  # ctx_id=0은 GPU 사용, ctx_id=-1은 CPU 사용

# NMS 임계값 설정
# - 낮출수록 더 많은 얼굴이 검출될 수 있지만 오탐률이 증가할 수 있음
app.det_model.nms_thresh = 0.6

# 이미지 파일 읽기
#img = cv2.imread("./faces/group_image.jpg")  # 처리할 이미지 파일의 경로로 변경하세요.
#img = cv2.imread("full-face-view.png")
#img = cv2.imread("F1.large.jpg")
img = cv2.imread(TARGET_IMG_FILE)
if img is None:
    raise FileNotFoundError(f"이미지를 불러올 수 없습니다. 경로를 확인하세요")

# 얼굴 검출 및 임베딩 추출
faces = app.get(img)

# 검출된 얼굴 처리
for idx, face in enumerate(faces):
    
    # 얼굴 영역 표시
    bbox = face.bbox.astype(int)
    #cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
    # 얼굴 임베딩 출력
    #print(f"얼굴 {idx+1} 임베딩 벡터:\n{face.embedding}")

    # YAW, PITCH, ROLL 계산
    yaw, pitch, roll = get_yaw_pitch_roll(face)

    # YAW, PITCH, ROLL 값 표시
    if yaw is not None and pitch is not None and roll is not None:
        cv2.putText(img, f"YAW: {yaw:.1f}", (bbox[0] + 5, bbox[1] + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (127, 127, 127), 1)
        cv2.putText(img, f"PITCH: {pitch:.1f}", (bbox[0] + 5, bbox[1] + 35), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (127, 127, 127), 1)
        cv2.putText(img, f"ROLL: {roll:.1f}", (bbox[0] + 5, bbox[1] + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (127, 127, 127), 1)

# 결과 이미지 표시
#cv2.imshow('Detection Result', img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

# 결과 저장
cv2.imwrite('detection_crooss.jpg', img)

In [ ]:
from tools import FaceSwapper, restore_face, upscale_image

- 이미지 전체 Upscaling

In [ ]:
import cv2

img = cv2.imread(TARGET_IMG_FILE)
# upscaling 함수 호출
upscaled_img = upscale_image(img, scale=4)

# 결과 저장
cv2.imwrite('upscaled_crooss.jpg', upscaled_img)

- 전체 얼굴 모두 Swap

In [ ]:
import cv2

face_swapper = FaceSwapper(det_size=(640, 640), nms_thresh=0.3)

# 소스 얼굴 이미지 로드
source_img = cv2.imread("test_hanni2.jpg")
    
# 소스 얼굴 설정 (face_index는 선택 사항)
success = face_swapper.set_source_face(source_img, face_index=0)
if not success:
    print("소스 얼굴 설정에 실패했습니다.")
    raise StopIteration  # 셀 실행을 중단
    
# 대상 이미지 로드
target_img = cv2.imread("upscaled_crooss.jpg")
    
# 얼굴 교체 수행 (ndarray 이미지를 입력으로 받아 결과를 ndarray로 반환)
swapped_img = face_swapper.swap_faces_in_image(target_img, draw_rectangle=True)

if swapped_img is not None:
    # 결과 이미지 표시
    #cv2.imshow('Result', swapped_img)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()    

    # 결과 저장
    cv2.imwrite('swapped_crooss.jpg', swapped_img)
else:
    print("얼굴 교체에 실패했습니다.")

- 전체 얼굴 모두 Restoring

In [ ]:
import cv2

# 대상 이미지 로드
target_img = cv2.imread("swapped_crooss.jpg")
    
# 얼굴 교체 수행 (ndarray 이미지를 입력으로 받아 결과를 ndarray로 반환)
restored_img = restore_face(target_img, draw_rectangle=True)

if restored_img is not None:
    # 결과 이미지 표시
    #cv2.imshow('Result', restored_img)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

    # 결과 저장
    cv2.imwrite('restored_crooss.jpg', restored_img)
else:
    print("얼굴 복구에 실패했습니다.")

C:\pypjt\face\Lib\site-packages\GFPGAN\experiments\pretrained_models


In [2]:
import cv2
import numpy as np
import torch
from gfpgan import GFPGANer

def restore_face_gfpgan(input_image: np.ndarray, upscale: int = 2, version: str = '1.4', use_gpu: bool = False, draw_rectangle: bool = True) -> np.ndarray:
    """
    GFPGAN을 사용하여 얼굴을 복원하는 함수

    :param input_image: ndarray 타입의 입력 이미지
    :param upscale: 업스케일링 배율 (기본값: 2)
    :param version: GFPGAN 모델 버전 ('1.3' 또는 '1.4')
    :param use_gpu: GPU 사용 여부 (기본값: False)
    :param draw_rectangle: 복원된 얼굴에 사각형을 그릴지 여부 (기본값: True)
    :return: ndarray 타입의 복원된 이미지
    """
    # 디바이스 설정
    device = 'cuda' if use_gpu and torch.cuda.is_available() else 'cpu'

    # 모델 파일 경로 설정
    if version == '1.4':
        model_path = './gfpgan/weights/GFPGANv1.4.pth'
    elif version == '1.3':
        model_path = 'experiments/pretrained_models/GFPGANv1.3.pth'
    else:
        raise ValueError('Invalid GFPGAN version. Choose "1.3" or "1.4".')

    # GFPGAN 초기화
    restorer = GFPGANer(
        model_path=model_path,
        upscale=upscale,
        arch='clean',
        channel_multiplier=2,
        bg_upsampler=None,
        device=device
    )

    # 얼굴 복원
    cropped_faces, restored_faces, restored_img = restorer.enhance(
        input_image,
        has_aligned=False,
        only_center_face=False,
        paste_back=True
    )

    # 복원된 얼굴에 사각형 그리기
    if draw_rectangle:
        for face in restorer.face_helper.det_faces:
            x1, y1, x2, y2, _ = face.astype(int)
            cv2.rectangle(restored_img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    return restored_img


In [5]:
import cv2

# 이미지 읽기
input_image = cv2.imread("swapped_crooss.jpg")

# 얼굴 복원
restored_image = restore_face_gfpgan(input_image, upscale=1, use_gpu=True, draw_rectangle=True)

# 결과 저장
cv2.imwrite('gfpgan_cross.png', restored_image)

c:\pypjt\face\Lib\site-packages\facexlib\parsing\__init__.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  load_net = torch.load(model_path, map_location=lambda storage,

FileNotFoundError: [Errno 2] No such file or directory: 'experiments/pretrained_models/GFPGANv1.4.pth'